#### 모델을 학습시키기 전에 의사결정

1. 어떤 loss를 쓸 것인지
2. learning rate은 몇으로 할 것인지
3. optimizer는 무엇으로 할 것인지
4. 학습 횟수는 몇 번으로 할 것인지

#### 기본루틴
- *optimizer.zero_grad()*  
- *pred = model(x)*  
- *loss = criterion(pred, x_labels)*  
- *loss.backward()*  
- *optimizer.step()*  

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

In [4]:
class MyModel(nn.Module):
    
    def __init__(self, X_dim, y_dim):
        super(MyModel, self).__init__()
        layer1 = nn.Linear(X_dim, 128)
        activation1 = nn.ReLU()
        layer2 = nn.Linear(128, y_dim)
        self.module = nn.Sequential(
            layer1,
            activation1,
            layer2
        )
        
    def forward(self, x):
        out = self.module(x)
        result = F.softmax(out, dim=1)
        return result        

In [5]:
# 준비재료
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-5
optimizer = optim.SGD(model.paraeters(), lr=learning_rate)
num_epochs = 2
num_batches = len(train_loader)

for epoch in range(num_epochs):
	for i, data in enumerate(train_loader):
		x, x_labels = data # x.size() = [batch, channel, x, y]
		# init grad
		optimizer.zero_grad() # step과 zero_grad는 쌍을 이루는 것이라고 생각하면 됨
		# forward
		pred = model(x)
		# calculate loss
		loss = criterion(pred, x_labels)
		# backpropagation
		loss.backward()
		# weight update
		optimizer.step()
		# 학습과정 출력
		running_loss += loss.item()
		if (i+1)%2000 == 0: # print every 2000 mini-batches
			print("epoch: {}/{} | step: {}/{} | loss: {:.4f}".format(epoch, num_epochs, i+1, num_batches, running_loss/2000))
			running_loss = 0.0

print("finish Training!")

NameError: name 'model' is not defined

## 단어의 표현 방법 https://wikidocs.net/60852


### 원핫 인코딩 한계
 1. 단어 개수가 늘어날수록 벡터 공간도 함께 늘어남(저장 공간 측면에서 매우 비효율적)
 2. 단어의 유사도를 표현하지 못함

### 희소 표현(희소 벡터)
 - 벡터 또는 행렬(matrix)의 값이 대부분 0으로 표현되는 방법을 희소 표현(sparse representation)이라고 함). 원-핫 벡터는 희소 벡터이다(원핫 인코딩의 한계를 그대로 가짐)
 
 Ex) 강아지 = [ 0 0 0 0 1 0 0 0 0 0 0 0 ... 중략 ... 0] # 이 때 1 뒤의 0의 수는 9995개. 차원은 10,000 (단어가 10,000개라고 가정 시)
 
### 밀집 표현(밀집 벡터)
 - 사용자가 밀집 표현의 차원을 128로 설정한다면, 모든 단어의 벡터 표현의 차원은 128로 바뀌면서 모든 값이 실수가 된다.
 
 Ex) 강아지 = [0.2 1.8 1.1 -2.1 1.1 2.8 ... 중략 ...] # 이 벡터의 차원은 128
 
### 워드 임베딩
 - 단어를 밀집 벡터(dense vector)의 형태로 표현하는 방법을 워드 임베딩이라고 함. 그리고 이 밀집 벡터를 워드임베딩 과정을 통해 나온 결과라고 하여 임베딩 벡터라고도 한다.
 
 - 워드 임베딩 방법론으로는 LSA, Word2Vec, FastText, Glove 등이 있음 
 - Pytorch 에서 제공하는 도구인 nn.embedding()는 앞서 언급한 방법들을 사용하지는 않지만, 단어를 랜덤한 값을 가지는 밀집 벡터로 변환한 뒤에, 인공 신경망의 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습하는 방법을 사용함
 
### 워드투벡터(Word2Vec)
 - 단어 간 유사도를 반영할 수 있도록 단어의 의미를 벡터화 하는 방법
 - 분산 표현 : 단어의 '의미'를 다차원 공간에 벡터화하는 방법
 - 이렇게 분산 표현을 이용하여 단어의 유사도를 벡터화하는 작업은 워드 임베딩(embedding) 작업에 속하기 때문에 이렇게 표현된 벡터 또한 임베딩 벡터(embedding vector)라고 하며, 저차원을 가지므로 바로 앞의 챕터에서 배운 밀집 벡터(dense vector)에도 속함
 
### 분산표현
 - 요약하면 희소 표현이 고차원에 각 차원이 분리된 표현 방법이었다면, 분산 표현은 저차원에 단어의 의미를 여러 차원에다가 분산하여 표현한다. 이런 표현 방법을 사용하면 단어 간 유사도를 계산할 수 있다.

 - 이를 위한 학습 방법으로는 NNLM, RNNLM 등이 있으나 요즘에는 해당 방법들의 속도를 대폭 개선시킨 Word2Vec가 많이 쓰이고 있다.

### PyTorch의 nn.Embedding()
- 파이토치에서는 임베딩 벡터를 사용하는 방법이 크게 두 가지가 있다. 
    - 임베딩 층(embedding layer)을 만들어 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법
    - 미리 사전에 훈련된 임베딩 벡터(pre-trained word embedding)들을 가져와 사용하는 방법